In [1]:
# Load libraries
import numpy as np
import pandas as pd
from timeit import default_timer

from sklearn.model_selection import train_test_split

In [2]:
dat = pd.read_csv('../data/dataset_mock_midterm.csv', sep = ",")

In [3]:
dat

,date,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm,exitus,dataset
0,2016-01,NaN,0.408730,12596.0,21.0,0.0,NaN,151.0,Q,1.0,0,train
1,2016-01,NaN,0.306931,20973.0,22.0,NaN,NaN,99.0,Q,1.0,0,train
2,2016-01,4.0,0.278481,19611.0,19.0,NaN,NaN,87.0,NaN,1.0,0,train
3,2016-01,3.0,0.150289,13583.0,22.0,NaN,NaN,100.0,Q,NaN,0,train
4,2016-01,1.0,0.016573,18042.0,2.0,NaN,NaN,44.0,Q,1.0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...
32701,2016-12,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0,0,test
32702,2016-12,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0,0,test
32703,2016-12,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN,0,test
32704,2016-12,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0,0,test


In [4]:
dat.drop('date', axis = 1, inplace = True)

In [5]:
cat_var = ['severity', 'ambulatory', 'origin', 'tip_grd', 'tip_adm']
non_cat_var = list(set(dat.columns) - set(cat_var))
num_var = list(set(dat.columns) - set(cat_var) - {'dataset', 'exitus'})

In [6]:
dat.isna().any()

severity            True
mortality_ratio     True
age                 True
num_proc            True
ambulatory          True
origin              True
expected_length     True
tip_grd             True
tip_adm             True
exitus             False
dataset            False
dtype: bool

In [7]:
from fancyimpute import IterativeImputer as MICE
# 3) Define "model"
model = MICE()

# 4) Train "model"
model.fit(dat[num_var][dat['dataset'] == 'train'])

# 5) "Predict"
dat[num_var] = model.transform(dat[num_var])
dat.isna().any()

severity            True
mortality_ratio    False
age                False
num_proc           False
ambulatory          True
origin              True
expected_length    False
tip_grd             True
tip_adm             True
exitus             False
dataset            False
dtype: bool

In [8]:
dat[cat_var] = dat[cat_var].astype('str')

In [9]:
dat.loc[dat['dataset'] == 'train', cat_var] = dat.loc[dat['dataset'] == 'train', cat_var].fillna('UNKNOWN')
dat[cat_var][dat['dataset'] == 'train'].isna().sum()

severity      0
ambulatory    0
origin        0
tip_grd       0
tip_adm       0
dtype: int64

In [10]:
dat.isna().any()

severity           False
mortality_ratio    False
age                False
num_proc           False
ambulatory         False
origin             False
expected_length    False
tip_grd            False
tip_adm            False
exitus             False
dataset            False
dtype: bool

In [11]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output = False, drop='first')

# 4) Training model
ohe.fit(dat[cat_var][dat['dataset'] == 'train'])

# 5) Predicting
dat_ohe = pd.DataFrame(ohe.fit_transform(dat[cat_var]))

# Optional
dat_ohe.columns = ohe.get_feature_names_out()
dat = pd.concat((dat[non_cat_var], dat_ohe), axis=1)

# Model 1 Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier as model_constructor_1
from sklearn.metrics import roc_auc_score as metric

In [13]:
criterion_values = ['gini', 'entropy']
max_depth_values = [5, 6, 7]
min_samples_split_values = [10, 20, 30]
min_samples_leaf_values = [29, 30, 31]
max_features_values = [None, 1, 2]

params_grid = {  'criterion': criterion_values,
                 'max_depth': max_depth_values,
                 'min_samples_split': min_samples_split_values,
                 'min_samples_leaf': min_samples_leaf_values,
                 'max_features': max_features_values}

In [14]:
n = len(params_grid['max_depth'])*len(params_grid['min_samples_split'])*len(params_grid['min_samples_leaf'])*len(params_grid['max_features'])*len(params_grid['criterion'])
print(str(n)+ ' iterations of Decision Tree')

162 iterations of Decision Tree


In [15]:
num_iter = 1
grid_results = pd.DataFrame(columns = ('criterion',
                                       'max_depth',
                                       'min_samples_split',
                                       'min_samples_leaf',
                                       'max_features',
                                       'auc_train',
                                       'auc_val',
                                       'time'))

for criterion in params_grid['criterion']:
    for max_depth in params_grid['max_depth']:
        for min_samples_split in params_grid['min_samples_split']:
            for min_samples_leaf in params_grid['min_samples_leaf']:
                for max_features in params_grid['max_features']:


                    # Start time
                    start_time = default_timer()

                    # Print trace
                    print('Iteracion = ' + str(num_iter))

                    # [3] Define model
                    model = model_constructor_1(criterion = criterion,
                                              max_depth = max_depth,
                                              min_samples_split = min_samples_split,
                                              min_samples_leaf = min_samples_leaf,
                                              max_features = max_features,
                                              random_state = 0)

                    # [4] Train model
                    model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1),
                                  dat[dat['dataset'] == 'train'].exitus.values)

                    # [5] Predict
                    pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!
                    pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!

                    # [6] Evaluate
                    metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
                    metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])

                    # Computational time
                    time = default_timer() - start_time

                    # print error
                    print('AUC train = %.2f - AUC validation = %.2f. Time spend = %.2f.'
                          % (metric_train, metric_val, time))

                    # Save iteration results
                    grid_results.loc[num_iter]=[criterion,
                                                max_depth,
                                                min_samples_split,
                                                min_samples_leaf,
                                                max_features,
                                             metric_train,
                                             metric_val,
                                            time]
                    num_iter += 1

print('Grid Search Total Computational Time: ', np.sum(grid_results.time.values))

Iteracion = 1
AUC train = 0.94 - AUC validation = 0.93. Time spend = 0.09.
Iteracion = 2
AUC train = 0.67 - AUC validation = 0.66. Time spend = 0.04.
Iteracion = 3
AUC train = 0.92 - AUC validation = 0.90. Time spend = 0.05.
Iteracion = 4
AUC train = 0.94 - AUC validation = 0.93. Time spend = 0.08.
Iteracion = 5
AUC train = 0.67 - AUC validation = 0.66. Time spend = 0.04.
Iteracion = 6
AUC train = 0.92 - AUC validation = 0.90. Time spend = 0.04.
Iteracion = 7
AUC train = 0.94 - AUC validation = 0.93. Time spend = 0.08.
Iteracion = 8
AUC train = 0.67 - AUC validation = 0.66. Time spend = 0.04.
Iteracion = 9
AUC train = 0.92 - AUC validation = 0.90. Time spend = 0.05.
Iteracion = 10
AUC train = 0.94 - AUC validation = 0.93. Time spend = 0.08.
Iteracion = 11
AUC train = 0.67 - AUC validation = 0.66. Time spend = 0.04.
Iteracion = 12
AUC train = 0.92 - AUC validation = 0.90. Time spend = 0.04.
Iteracion = 13
AUC train = 0.94 - AUC validation = 0.93. Time spend = 0.10.
Iteracion = 14
AUC tr

In [16]:
grid_results = grid_results.sort_values(by = ['auc_val', 'auc_train', 'time'], ascending = [False, False, True])
grid_results

,criterion,max_depth,min_samples_split,min_samples_leaf,max_features,auc_train,auc_val,time
49,gini,6,30,30,None,0.946581,0.935483,0.085252
40,gini,6,20,30,None,0.946581,0.935483,0.086896
31,gini,6,10,30,None,0.946581,0.935483,0.089045
52,gini,6,30,31,None,0.946536,0.935474,0.086294
43,gini,6,20,31,None,0.946536,0.935474,0.087315
...,...,...,...,...,...,...,...,...
23,gini,5,30,30,1,0.669191,0.656327,0.039543
14,gini,5,20,30,1,0.669191,0.656327,0.039833
5,gini,5,10,30,1,0.669191,0.656327,0.040977
11,gini,5,20,29,1,0.669191,0.656327,0.041439


In [17]:
best_model = grid_results.iloc[0]
best_model

criterion                gini
max_depth                   6
min_samples_split          30
min_samples_leaf           30
max_features             None
auc_train            0.946581
auc_val              0.935483
time                 0.085252
Name: 49, dtype: object

In [18]:
model  = model_constructor_1(criterion = best_model['criterion'],
                                              max_depth = best_model['max_depth'],
                                              min_samples_split = best_model['min_samples_split'],
                                              min_samples_leaf = best_model['min_samples_leaf'],
                                              max_features = best_model['max_features'],
                                              random_state = 0)

In [19]:
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=30, min_samples_split=30,
                       random_state=0)

In [20]:
# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

In [21]:
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.95 - Metric val = 0.94 - Metric test = 0.93


# Model 2 Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier as model_constructor_2

In [23]:
# Random Forest
n_estimators_values = [10, 100, 1000]
max_features_values = [2, 5, 10]
max_samples_values = [100, 1000, dat[dat['dataset'] == 'train'].shape[0]]

params_grid = {'max_features': max_features_values,
              'n_estimators': n_estimators_values,
               'max_samples': max_samples_values}

In [24]:
num_iter = 1
grid_results = pd.DataFrame(columns = ('max_features',
                                       'n_estimators',
                                       'max_samples',
                                       'metric_train',
                                       'metric_val'))

for max_features in params_grid['max_features']:
    for n_estimators in params_grid['n_estimators']:
        for max_samples in params_grid['max_samples']:

                        # Print trace
                        print('Iteracion = ' + str(num_iter))

                        # [3] Define model
                        model = model_constructor_2(max_features = max_features,
                                                  n_estimators = n_estimators,
                                                  max_samples = max_samples,
                                                  random_state = 0)

                        # [4] Train model
                        model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1),
                                  dat[dat['dataset'] == 'train'].exitus.values)


                        # [5] Predict
                        pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
                        pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!

                        # [6] Compute metric
                        metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
                        metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])

                        # print error
                        print('Metric train = %.2f - Metric validation = %.2f.'
                              % (metric_train, metric_val))

                        # Save iteration results
                        grid_results.loc[num_iter]=[ max_features,
                                                    n_estimators,
                                                    max_samples,
                                                 metric_train,
                                                 metric_val]
                        num_iter += 1



Iteracion = 1
Metric train = 0.84 - Metric validation = 0.82.
Iteracion = 2
Metric train = 0.90 - Metric validation = 0.85.
Iteracion = 3
Metric train = 1.00 - Metric validation = 0.84.
Iteracion = 4
Metric train = 0.93 - Metric validation = 0.93.
Iteracion = 5
Metric train = 0.96 - Metric validation = 0.93.
Iteracion = 6
Metric train = 1.00 - Metric validation = 0.92.
Iteracion = 7
Metric train = 0.94 - Metric validation = 0.93.
Iteracion = 8
Metric train = 0.97 - Metric validation = 0.94.
Iteracion = 9
Metric train = 1.00 - Metric validation = 0.93.
Iteracion = 10
Metric train = 0.86 - Metric validation = 0.85.
Iteracion = 11
Metric train = 0.90 - Metric validation = 0.86.
Iteracion = 12
Metric train = 1.00 - Metric validation = 0.86.
Iteracion = 13
Metric train = 0.93 - Metric validation = 0.92.
Iteracion = 14
Metric train = 0.96 - Metric validation = 0.94.
Iteracion = 15
Metric train = 1.00 - Metric validation = 0.92.
Iteracion = 16
Metric train = 0.94 - Metric validation = 0.93.
I

In [25]:
grid_results = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model = grid_results.iloc[0]
best_model

max_features      10.000000
n_estimators    1000.000000
max_samples     1000.000000
metric_train       0.961979
metric_val         0.937384
Name: 26, dtype: float64

In [26]:
# Extract variable importance (tree-based)
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)
var_imp = pd.DataFrame({'var': dat.drop(['exitus', 'dataset'], axis = 1).columns, 'imp': model.feature_importances_})
var_imp.sort_values(['imp'], ascending = False, inplace = True)
var_imp

,var,imp
1,mortality_ratio,3.213752e-01
0,age,2.589571e-01
3,expected_length,1.476638e-01
2,num_proc,1.153729e-01
18,tip_grd_nan,2.751719e-02
16,origin_nan,2.720553e-02
21,tip_adm_nan,2.160280e-02
6,severity_4.0,1.794256e-02
7,severity_nan,1.489820e-02
5,severity_3.0,1.338546e-02


In [27]:
dat = dat[['mortality_ratio', 'age', 'expected_length', 'num_proc', 'exitus', 'dataset']] # This is just a fake example of how to select the most important variables.

In [28]:
# [4] Train model
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)


# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

In [29]:
# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 1.00 - Metric val = 0.92 - Metric test = 0.93


# Model 3 XGBoost

In [30]:
from xgboost import XGBClassifier as model_constructor_3

model = model_constructor_3(early_stopping_rounds=10,
                            n_estimators=1000,
                            eval_metric="auc",
                            random_state=1)

In [31]:
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1),
          np.array(dat[dat['dataset'] == 'train'].exitus.values),
          eval_set=[(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'val'].exitus.values)], 
          verbose=True,
          )

[0]	validation_0-auc:0.92578
[1]	validation_0-auc:0.92765
[2]	validation_0-auc:0.92970
[3]	validation_0-auc:0.93286
[4]	validation_0-auc:0.93248
[5]	validation_0-auc:0.93186
[6]	validation_0-auc:0.93142
[7]	validation_0-auc:0.93240
[8]	validation_0-auc:0.93261
[9]	validation_0-auc:0.93175
[10]	validation_0-auc:0.93247
[11]	validation_0-auc:0.93232
[12]	validation_0-auc:0.93241


J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dt

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [32]:
pred_train_p = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1))
pred_val_p = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1))
pred_test_p = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1))
# Calcular métricas de evaluación
auc_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train_p[:,1])
auc_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val_p[:,1])
auc_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test_p[:,1])
results = pd.DataFrame()

new_data = pd.DataFrame(data={'model': ['XGBoost (Default)'], 'auc_train': [auc_train], 'auc_val': [auc_val], 'auc_test': [auc_test]}, columns=['model', 'auc_train', 'auc_val', 'auc_test'])

results = pd.concat([results, new_data], ignore_index=True)

results

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dt

,model,auc_train,auc_val,auc_test
0,XGBoost (Default),0.954507,0.932856,0.934233


# Model 4 SVM

In [33]:
from sklearn.svm import SVC as model_constructor_4

In [34]:
d = dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1).shape[1]
m = np.mean(dat[dat['dataset'] == 'train'].exitus.values)
s = np.std(dat[dat['dataset'] == 'train'].exitus.values)

In [35]:
C_cherk = np.max([np.abs(m + 3*s),np.abs(m - 3*s)])
gamma_cherk = np.power(0.2, 1/d)

In [36]:
# SVM
C_values = [0.1, 1, 10]
gamma_values = [0.01, 1, 100]

params_grid = {'C': C_values,
               'gamma': gamma_values}

In [37]:
num_iter = 1
grid_results = pd.DataFrame(columns = ('C',
                                       'gamma',
                                       'auc_train',
                                       'auc_val',
                                       'time'))

for C in params_grid['C']:
    for gamma in params_grid['gamma']:

                    # Start time
                    start_time = default_timer()

                    # Print trace
                    print('Iteracion = ' + str(num_iter))

                    # [3] Define model
                    model = model_constructor_4(C = C,
                                              gamma = gamma,
                                              probability = True,
                                              random_state = 0) # Probability = True!!!

                    # [4] Train model
                    model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)

                    # [5] Predict
                    pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!
                    pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!

                    # [6] Compute metric
                    metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,0])
                    metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,0])

                    # Computational time
                    time = default_timer() - start_time

                    # print error
                    print('AUC train = %.2f - AUC validation = %.2f. Time spend = %.2f.'
                          % (metric_train, metric_val, time))

                    # Save iteration results
                    grid_results.loc[num_iter]=[C,
                                                gamma,
                                                metric_train,
                                                metric_val,
                                                time]
                    num_iter += 1

print('Grid Search Total Computational Time: ', np.sum(grid_results.time.values))

Iteracion = 1
AUC train = 0.03 - AUC validation = 0.45. Time spend = 171.62.
Iteracion = 2
AUC train = 0.00 - AUC validation = 0.33. Time spend = 532.83.
Iteracion = 3
AUC train = 0.00 - AUC validation = 0.49. Time spend = 594.49.
Iteracion = 4
AUC train = 0.03 - AUC validation = 0.45. Time spend = 300.82.
Iteracion = 5
AUC train = 0.00 - AUC validation = 0.32. Time spend = 714.31.
Iteracion = 6
AUC train = 0.00 - AUC validation = 0.49. Time spend = 784.88.
Iteracion = 7
AUC train = 0.98 - AUC validation = 0.51. Time spend = 283.18.
Iteracion = 8
AUC train = 0.00 - AUC validation = 0.32. Time spend = 747.88.
Iteracion = 9
AUC train = 0.00 - AUC validation = 0.49. Time spend = 757.84.
Grid Search Total Computational Time:  4887.841272299993


In [38]:
grid_results = grid_results.sort_values(by = ['auc_val', 'auc_train', 'time'], ascending = [False, False, True])
grid_results

,C,gamma,auc_train,auc_val,time
7,10.0,0.01,0.980731,0.508097,283.183016
9,10.0,100.00,0.000000,0.488844,757.837903
6,1.0,100.00,0.000000,0.488810,784.881326
3,0.1,100.00,0.000000,0.488769,594.486062
4,1.0,0.01,0.031175,0.453393,300.823236
1,0.1,0.01,0.034346,0.452615,171.616194
2,0.1,1.00,0.000000,0.326914,532.830954
5,1.0,1.00,0.000000,0.324975,714.305631
8,10.0,1.00,0.000000,0.324802,747.876950


In [39]:
best_model = grid_results.iloc[0]
best_model

C             10.000000
gamma          0.010000
auc_train      0.980731
auc_val        0.508097
time         283.183016
Name: 7, dtype: float64

In [40]:
print('Old train data size = ' + str(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1).shape))
print('Old train target size = ' + str(dat[dat['dataset'] == 'train'].exitus.values.shape))

# Combine train and validación
X_train = np.concatenate((dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)), axis = 0)
y_train = np.concatenate((dat[dat['dataset'] == 'train'].exitus.values, dat[dat['dataset'] == 'val'].exitus.values), axis = 0)

print('New train data size = ' + str(X_train.shape))
print('New train target size = ' + str(y_train.shape))

Old train data size = (22894, 4)
Old train target size = (22894,)
New train data size = (27799, 4)
New train target size = (27799,)


In [42]:
# [3] Define model
model = model_constructor_4(C = best_model.C,
                          gamma = best_model.gamma,
                          probability = True,
                          random_state = 0) # probability = True!!!

# [4] Train model
model.fit(X_train, y_train)

# [5] Predict
pred_train = model.predict_proba(X_train)
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1))

# [6] Compute metric
metric_train = metric(y_train, pred_train, multi_class = 'ovo')
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test,multi_class = 'ovo')

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


ValueError: y should be a 1d array, got an array of shape (27799, 2) instead.

In [ ]:
# print error
print('AUC train = %.2f - AUC test = %.2f'
      % (metric_train, metric_test))